# 1. Import the necessary libraries for you to be able to query 4square API

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [7]:
# I could only get this by adding coordinates. 
url = "https://api.foursquare.com/v3/places/search?ll=40.39353,-3.69871&query=coffee&categories=13032&sort=DISTANCE&radius=50000&limit=10"

with open('C:/Users/joice/OneDrive/Documentos/Ironhack/ML course/secrets/secrets.txt', 'r') as file:
    api_key = file.read().split('fsq-token:')[1].strip()

headers = {
    "accept": "application/json",
    "Authorization": f"{api_key}"
}

response = requests.get(url, headers=headers)

data = response.json()

coffee_places = []
if 'results' in data:
    for place in data['results']:
        coffee_places.append({
            'name': place['name'],
            'latitude': place['geocodes']['main']['latitude'],
            'longitude': place['geocodes']['main']['longitude']
        })

coffees_near = pd.DataFrame(coffee_places)

print(coffees_near)


                           name   latitude  longitude
0                  Dot Café Bar  40.396051  -3.698800
1              BBIG Delice Café  40.396268  -3.693275
2  Sanpas Cafetería Restaurante  40.390827  -3.689063
3                Formula Nietos  40.404251  -3.702233
4                   Hola Coffee  40.407002  -3.699084
5                El Rincón Guay  40.407447  -3.704102
6                  Pum Pum Café  40.408108  -3.701986
7                      Cafelito  40.408404  -3.702780
8           Nuevo Café Barbieri  40.409294  -3.701103
9                      Scarlett  40.408871  -3.692709


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [8]:
# I changed the limit to 5 in the url

url = "https://api.foursquare.com/v3/places/search?ll=40.39353,-3.69871&query=coffee&categories=13032&sort=DISTANCE&radius=50000&limit=5"

with open('C:/Users/joice/OneDrive/Documentos/Ironhack/ML course/secrets/secrets.txt', 'r') as file:
    api_key = file.read().split('fsq-token:')[1].strip()

headers = {
    "accept": "application/json",
    "Authorization": f"{api_key}"
}

response = requests.get(url, headers=headers)

data = response.json()

coffee_places = []
if 'results' in data:
    for place in data['results']:
        coffee_places.append({
            'name': place['name'],
            'latitude': place['geocodes']['main']['latitude'],
            'longitude': place['geocodes']['main']['longitude']
        })

coffees_near = pd.DataFrame(coffee_places)

print(coffees_near)


                           name   latitude  longitude
0                  Dot Café Bar  40.396051  -3.698800
1              BBIG Delice Café  40.396268  -3.693275
2  Sanpas Cafetería Restaurante  40.390827  -3.689063
3                Formula Nietos  40.404251  -3.702233
4                   Hola Coffee  40.407002  -3.699084


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [9]:
def foursquare_places (venue, coordinates):
    url = f"https://api.foursquare.com/v3/places/search?ll={coordinates}&query={venue}&sort=DISTANCE&limit=10"


    with open('C:/Users/joice/OneDrive/Documentos/Ironhack/ML course/secrets/secrets.txt', 'r') as file:
        api_key = file.read().split('fsq-token:')[1].strip()

    headers = {
        "accept": "application/json",
        "Authorization": f"{api_key}"
    }
    
    response = requests.get(url, headers=headers)
    search = response.json()
    
    places = []
    if 'results' in search:
        for place in search['results'][:10]: 
            places.append({
                'name': place['name'],
                'latitude': place['geocodes']['main']['latitude'],
                'longitude': place['geocodes']['main']['longitude']
            })

    # Convert to DataFrame and return it
    return pd.DataFrame(places)





In [10]:
foursquare_places("Starbucks", "41.387,2.1701")

,name,latitude,longitude
0,Starbucks,41.387848,2.169659
1,Starbucks,41.388096,2.168053
2,Starbucks,41.385827,2.167881
3,Starbucks,41.388798,2.172099
4,Starbucks,41.383598,2.171119
5,Starbucks,41.381925,2.172230
6,Starbucks,41.390753,2.165031
7,Starbucks,41.385252,2.163222
8,Starbucks,41.385000,2.177103
9,Starbucks,41.382413,2.176695


In [11]:
foursquare_places("Restaurants", "41.387,2.1701")

,name,latitude,longitude
0,Hard Rock Cafe,41.385897,2.170779
1,Honest Greens Plaça Catalunya,41.387526,2.167989
2,La Esquina,41.385888,2.167912
3,Teresa Carles,41.385065,2.168022
4,Macchina,41.386481,2.166706
5,El Asador de Aranda,41.389233,2.172018
6,Mazah,41.387752,2.166598
7,Ciutat Comtal,41.388858,2.166857
8,Choko,41.388406,2.173858
9,Citizen Café,41.389412,2.172787


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [12]:
Bcn_starbucks = foursquare_places("Starbucks", "41.387,2.1701")
Bcn_starbucks.to_json("C:/Users/joice/OneDrive/Documentos/Ironhack/ML course/Labs w7/lab-apis/downloaded from labs/starbucks_places.json", orient = "records", indent=4)

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [13]:
# your code here